In [1]:
import json
with open('/home/mehdi/invariance_data/Vert/Mask_Vert/Mask_LOC_IMG.json', 'r') as f:
        data = json.load(f)#
#with open('/home/mehdi/Evaluation_detection/coco/Correct/coco_GT.json', 'r') as f:
#    refined_data = json.load(f) instances_val2017_invariance_vert_flip
#with open('/home/mehdi/Evaluation_detection/coco/instances_val2017.json', 'r') as f:
#        data_valid = json.load(f) coco_invariance_vertical_GT
with open('/home/mehdi/invariance_data/annotations/coco_invariance_vertical_GT.json', 'r') as f:
    Img_valid = json.load(f)


In [2]:
# Error type -localization

refined_data = []
scores = []

for det  in data:
    scores.append(det['score'])

sorted_mylist = sorted(((v,i) for i,v in enumerate(scores)), reverse = True)  
for index in sorted_mylist:
    max_iou = 0
    max_iou_2 = 0
    detections = data[index[1]]
    Flag = False
    for gt_index,annot in enumerate(data_valid[str(detections['image_id'])]):
        if (annot ['category_id'] == detections ['category_id']):
            x_left = max (annot['bbox'][0], detections ['bbox'][0])
            y_top = max (annot['bbox'][1],detections ['bbox'][1])
            x_right = min(annot['bbox'][0]+annot['bbox'][2],detections ['bbox'][0]+detections ['bbox'][2])
            y_bottom = min (annot['bbox'][1]+annot['bbox'][3],detections ['bbox'][1]+detections ['bbox'][3])
            intersect = (x_right-x_left)*(y_bottom-y_top)
            target_area = annot['bbox'][2]* annot['bbox'][3]
            detect_area = detections ['bbox'][2]*detections ['bbox'][3]
            iou = intersect/float(target_area +detect_area -intersect)
            if (iou > max_iou and not 'localization' in data_valid[str(detections['image_id'])][gt_index].keys()):
                max_iou = iou
                updated_index = gt_index
                Flag=True
            else:
                if (iou > max_iou_2):
                    max_iou_2 = iou
                    updated_index_2 = gt_index



    if Flag:
        if (max_iou> 0.1 and max_iou <0.5): 
            corrected_bbox = {}
            corrected_bbox['image_id'] = detections['image_id']
            corrected_bbox ['score'] = detections['score']
            corrected_bbox['category_id'] = detections['category_id']
            corrected_bbox['bbox'] = data_valid[str(detections['image_id'])][updated_index]['bbox']
            data_valid[str(detections['image_id'])][updated_index]['localization']=1
            refined_data.append (corrected_bbox)
        elif(max_iou>= 0.5):
            
            refined_data.append(detections)
            data_valid[str(detections['image_id'])][updated_index]['localization']=1
 
    else:
        
        if (max_iou_2> 0.1 and max_iou_2 <0.5):
            
            corrected_bbox = {}
            corrected_bbox['image_id'] = detections['image_id']
            corrected_bbox ['score'] = detections['score']
            corrected_bbox['category_id'] = detections['category_id']
            corrected_bbox['bbox'] = data_valid[str(detections['image_id'])][updated_index_2]['bbox']
            refined_data.append (corrected_bbox)
        elif (max_iou_2>=0.5):
            refined_data.append(detections)
        


        

8400
95082
132114
8421
24459
12573
165881
62399
103482


In [2]:
# Duplicates
nms_data= []

for images in Img_valid:
    for gtboxes in Img_valid [images]:  
        if (not 'seen_flag' in gtboxes.keys()):
            scores = []
            for det  in data[images]:
                scores.append(det['score'])
            sorted_mylist = sorted(((v,i) for i,v in enumerate(scores)), reverse = True)  
            for index in sorted_mylist:
                maxiou_sofar = 0
                detections = data[images][index[1]]
                if (gtboxes ['category_id']== detections['category_id']):
                    x_left = max (gtboxes['bbox'][0], detections ['bbox'][0])
                    y_top = max (gtboxes['bbox'][1],detections ['bbox'][1])
                    x_right = min(gtboxes['bbox'][0]+gtboxes['bbox'][2],detections ['bbox'][0]+detections ['bbox'][2])
                    y_bottom = min (gtboxes['bbox'][1]+gtboxes['bbox'][3],detections ['bbox'][1]+detections ['bbox'][3])
                    intersect = (x_right-x_left)*(y_bottom-y_top)
                    target_area = gtboxes['bbox'][2]* gtboxes['bbox'][3]
                    detect_area = detections ['bbox'][2]*detections ['bbox'][3]
                    iou = intersect/float(target_area +detect_area -intersect)
                    if (not 'flag_dup' in detections.keys()): 
                        maxiou_sofar = iou
                        max_iou = iou
                        for gt_index,otherbboxes in enumerate (Img_valid [images]):
                            if (not 'seen_flag' in otherbboxes.keys() and otherbboxes!= gtboxes and otherbboxes['category_id'] == detections['category_id']):
                                x_left = max (otherbboxes['bbox'][0], detections ['bbox'][0])
                                y_top = max (otherbboxes['bbox'][1],detections ['bbox'][1])
                                x_right = min(otherbboxes['bbox'][0]+otherbboxes['bbox'][2],detections ['bbox'][0]+detections ['bbox'][2])
                                y_bottom = min (otherbboxes['bbox'][1]+otherbboxes['bbox'][3],detections ['bbox'][1]+detections ['bbox'][3])
                                intersect = (x_right-x_left)*(y_bottom-y_top)
                                target_area = otherbboxes['bbox'][2]* otherbboxes['bbox'][3]
                                detect_area = detections ['bbox'][2]*detections ['bbox'][3]
                                iou = intersect/float(target_area +detect_area -intersect)
                                if (iou > max_iou and iou >= 0.5): 
                                    max_iou = iou
                                    updated_gtindex = gt_index
                        if (max_iou>maxiou_sofar): 
                            Img_valid [images][updated_gtindex]['seen_flag'] = 1
                            nms_data.append(detections)
                            detections['flag_dup']=1
                        elif (max_iou <= maxiou_sofar and maxiou_sofar >=0.5):
                            gtboxes['seen_flag']  =1
                            nms_data.append(detections)
                            detections['flag_dup']=1
                            break
                        
                                                 

25391
5000


In [2]:
# Add_misses
nms_data= []

for images in Img_valid:
    for gtboxes in Img_valid [images]:  
        if (not 'seen_flag' in gtboxes.keys()):
            scores = []
            for det  in data[images]:
                scores.append(det['score'])
            sorted_mylist = sorted(((v,i) for i,v in enumerate(scores)), reverse = True)  
            for index in sorted_mylist:
                maxiou_sofar = 0
                detections = data[images][index[1]]
                if (gtboxes ['category_id']== detections['category_id']):
                    x_left = max (gtboxes['bbox'][0], detections ['bbox'][0])
                    y_top = max (gtboxes['bbox'][1],detections ['bbox'][1])
                    x_right = min(gtboxes['bbox'][0]+gtboxes['bbox'][2],detections ['bbox'][0]+detections ['bbox'][2])
                    y_bottom = min (gtboxes['bbox'][1]+gtboxes['bbox'][3],detections ['bbox'][1]+detections ['bbox'][3])
                    intersect = (x_right-x_left)*(y_bottom-y_top)
                    target_area = gtboxes['bbox'][2]* gtboxes['bbox'][3]
                    detect_area = detections ['bbox'][2]*detections ['bbox'][3]
                    iou = intersect/float(target_area +detect_area -intersect)
                    if (not 'flag_det' in detections.keys()): 
                        maxiou_sofar = iou
                        max_iou = iou
                        for gt_index,otherbboxes in enumerate (Img_valid [images]):
                            if (not 'seen_flag' in otherbboxes.keys() and otherbboxes!= gtboxes and otherbboxes['category_id'] == detections['category_id']):
                                x_left = max (otherbboxes['bbox'][0], detections ['bbox'][0])
                                y_top = max (otherbboxes['bbox'][1],detections ['bbox'][1])
                                x_right = min(otherbboxes['bbox'][0]+otherbboxes['bbox'][2],detections ['bbox'][0]+detections ['bbox'][2])
                                y_bottom = min (otherbboxes['bbox'][1]+otherbboxes['bbox'][3],detections ['bbox'][1]+detections ['bbox'][3])
                                intersect = (x_right-x_left)*(y_bottom-y_top)
                                target_area = otherbboxes['bbox'][2]* otherbboxes['bbox'][3]
                                detect_area = detections ['bbox'][2]*detections ['bbox'][3]
                                iou = intersect/float(target_area +detect_area -intersect)
                                if (iou > max_iou and iou >= 0.5): 
                                    max_iou = iou
                                    updated_gtindex = gt_index
                        if (max_iou>maxiou_sofar): 
                            Img_valid [images][updated_gtindex]['seen_flag'] = 1
                            corrected_box = {}
                            corrected_box['bbox']= Img_valid [images][updated_gtindex]['bbox']
                            corrected_box['category_id'] = Img_valid [images][updated_gtindex]['category_id']
                            corrected_box['score'] =detections ['score']
                            corrected_box ['image_id']= Img_valid [images][updated_gtindex] ['image_id']
                            nms_data.append(corrected_box) 
                            detections['flag_det']=1
                        elif (max_iou <= maxiou_sofar and maxiou_sofar >=0.5):
                            gtboxes['seen_flag']  =1
                            corrected_box = {}
                            corrected_box['bbox']= gtboxes['bbox']
                            corrected_box['category_id'] = gtboxes['category_id']
                            corrected_box['score'] =detections ['score']
                            corrected_box ['image_id']= gtboxes['image_id']
                            nms_data.append(corrected_box) 
                            detections['flag_det']=1
                            break
                        
                                                                         
for images in Img_valid:
    for gtboxes in Img_valid [images]:  
        if (not 'seen_flag' in gtboxes.keys()):
            missed_detection = {}
            missed_detection['bbox']= gtboxes['bbox']
            missed_detection['category_id'] = gtboxes['category_id']
            missed_detection['score'] =1
            missed_detection ['image_id']= gtboxes ['image_id']
            nms_data.append (missed_detection)


36781
11390
5000


In [4]:
# Error confusion with background
refined_data = []
scores = []

for det  in data:
    scores.append(det['score'])

sorted_mylist = sorted(((v,i) for i,v in enumerate(scores)), reverse = True)  
for index in sorted_mylist:
    max_iou = 0
    max_iou_2 = 0
    detections = data[index[1]]
    Flag = False
    for gt_index,annot in enumerate(data_valid[str(detections['image_id'])]):
        if (annot ['category_id'] == detections ['category_id']):
            x_left = max (annot['bbox'][0], detections ['bbox'][0])
            y_top = max (annot['bbox'][1],detections ['bbox'][1])
            x_right = min(annot['bbox'][0]+annot['bbox'][2],detections ['bbox'][0]+detections ['bbox'][2])
            y_bottom = min (annot['bbox'][1]+annot['bbox'][3],detections ['bbox'][1]+detections ['bbox'][3])
            intersect = (x_right-x_left)*(y_bottom-y_top)
            target_area = annot['bbox'][2]* annot['bbox'][3]
            detect_area = detections ['bbox'][2]*detections ['bbox'][3]
            iou = intersect/float(target_area +detect_area -intersect)
            if (iou > max_iou and not 'background' in data_valid[str(detections['image_id'])][gt_index].keys()):
                max_iou = iou
                updated_index = gt_index
                Flag=True
            else:
                if (iou > max_iou_2):
                    max_iou_2 = iou
                    updated_index_2 = gt_index



    if Flag:
        if ((max_iou> 0.1 and max_iou <0.5) or max_iou>= 0.5): 
            
            data_valid[str(detections['image_id'])][updated_index]['background']=1
            refined_data.append (detections)     
            
    else:
    
        if ((max_iou_2> 0.1 and max_iou_2 <0.5) or max_iou_2>=0.5):
            refined_data.append (detections)
       

8400
95082
132114
8421
24459
12573
165881
62399
103482


In [5]:
#convert the json file to Img based 
Detect_imgkey = {}
print (len(data))
count = 0
for img in data_valid['images']:
    img_detections= []
    for detections in data:
        if (detections['image_id']==img['id']):
            count +=1
            img_detections.append(detections)
    Detect_imgkey[img['id']]=img_detections
print(count)

62399
62399


In [5]:
GT_table = {}
for img in data_valid['images']:
    img_bboxes = []
    for bboxes in data_valid['annotations']:
        if (bboxes['image_id']==img['id']):
            img_bboxes.append(bboxes)
    GT_table[img['id']]=img_bboxes